# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key
gmaps.configure(api_key = g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load csv file

weather_csv = "Data/weather_data.csv"

weather_data = pd.read_csv(weather_csv)
weather_data.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Punta Arenas,75,CL,1584464171,54,-53.15,-70.92,53.60,27.07
1,Tessalit,35,ML,1584464285,10,20.20,1.01,92.21,16.02
2,Klaksvík,23,FO,1584464285,69,62.23,-6.59,39.20,17.22
3,Cape Town,13,ZA,1584463861,73,-33.93,18.42,70.00,16.11
4,Neuquén,0,AR,1584464131,27,-38.95,-68.06,78.80,4.70


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Store latitude and longitude in locations
locations = weather_data[["Lat","Lng"]]

#store humidity

humidity = weather_data["Humidity"].astype(float)

humidity



0      54.0
1      10.0
2      69.0
3      73.0
4      27.0
       ... 
572    36.0
573    65.0
574    92.0
575    39.0
576    82.0
Name: Humidity, Length: 577, dtype: float64

In [4]:
# Plot heat map
fig = gmaps.figure()

# create heat layer
heat = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False,
                          max_intensity=10, point_radius=1)

fig.add_layer(heat)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
new_df = pd.DataFrame(weather_data, columns = ["City","Max Temp","Wind Speed", "Cloudiness"])
new_df
max_temp = (new_df["Max Temp"] <= 80) & (new_df["Max Temp"] >= 70)
wind_speed = (new_df["Wind Speed"] <= 10)
cloudiness = (new_df["Cloudiness"] == 0)

new_df[max_temp & wind_speed & cloudiness]

,City,Max Temp,Wind Speed,Cloudiness
4,Neuquén,78.80,4.70,0
7,Chabahar,71.33,4.05,0
50,Marsh Harbour,73.31,8.90,0
142,Lebu,70.07,9.31,0
181,Gizo Government Station,77.09,4.72,0
196,Phulbāni,75.83,4.27,0
201,The Valley,78.53,9.08,0
218,Khajuraho,74.93,4.23,0
362,Naliya,70.61,2.53,0
463,Manohar Thāna,73.13,9.71,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
# Make a hotel dataframe and add the correct columns

hotel_df = pd.DataFrame(weather_data[["City","Country","Lat","Lng"]])

# add an empty column for the Hotels
hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Punta Arenas,CL,-53.15,-70.92,
1,Tessalit,ML,20.20,1.01,
2,Klaksvík,FO,62.23,-6.59,
3,Cape Town,ZA,-33.93,18.42,
4,Neuquén,AR,-38.95,-68.06,
...,...,...,...,...,...
572,Laas,IT,46.62,10.70,
573,Darhan,MN,49.49,105.92,
574,Green River,US,41.53,-109.47,
575,Beipiao,CN,41.79,120.78,


In [7]:
# search for the hotels
g_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

hotel_name = []

params = {
    "radius":5000,
    "types":"hotel",
    "keyword":"hotel",
    "key":g_key
}


for index,row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # request api
    name_address = requests.get(g_url, params=params)
    
    #conver to json
    name_address = name_address.json()
    
    try:
        hotel_name.append(name_address["results"][0]["name"])
        
    except:
        hotel_name.append(np.nan)


KeyboardInterrupt: 

In [ ]:
hotel_name

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

# Display Map